In [ ]:
from math import sin, cos, pi     #数学関数・定数のインポート
import numpy as np                #数値計算機能のインポート 別名np
import matplotlib.pyplot as plt   #グラフ描画機能のインポート 別名plt
from mpl_toolkits.mplot3d import Axes3D #3次元描画機能のインポート
import ipywidgets as widgets      #対話的処理機能のインポート

# ロボット・マニピュレータ

### 同次変換行列(3次元)を作る関数

In [ ]:
def H_3d(angle, axis, r):

  a1, a2, a3 = axis #回転軸の成分
  aa1 = a1*a1 #2乗を作っておく
  aa2 = a2*a2
  aa3 = a3*a3
  C = cos(angle)
  S = sin(angle)

  H = np.array([
    [ aa1 + (1-aa1)*C,    a1*a2*(1-C) - a3*S, a1*a3*(1-C) + a2*S, r[0] ],
    [ a1*a2*(1-C) + a3*S, aa2 + (1-aa2)*C,    a2*a3*(1-C) - a1*S, r[1] ],
    [ a1*a3*(1-C) - a2*S, a2*a3*(1-C) + a1*S, aa3 + (1-aa3)*C,    r[2] ],
    [                  0,                  0,               0,      1  ],
  ])

  return H

In [ ]:
H_3d(pi/2, [1,0,0], [1,0,0]) #お試し

※ `6.123234e-17` $=6.123234\times10^{-17}$なので，計算機誤差を含む 0 のこと．

### ダミー成分を追加・除去する関数（3次元）

In [ ]:
### 物理座標にダミー成分を追加する関数
def add_dummy(x):
#  return np.array([x[0], x[1], x[2], 1]) #ダミー成分を追加
  return np.append(x, 1) #同じ処理のPython的な書き方

### 同次座標からダミー変数を除去する関数
def remove_dummy(xx):
#  return np.array([xx[0], xx[1], xx[2]]) #ダミー成分を除去
  return xx[:-1] #同じ処理のPython的な書き方

In [ ]:
H = H_3d(pi/2, [1,0,0], [1,2,3]) #お試し
x = np.array([1,2,3])
print(x)
hx = add_dummy(x)
print(hx)
hy = np.dot(H, hx) #同次変換行列と同次座標の積
print(hy)
y = remove_dummy(hy)
print(y)

### テキストpart2.pdf の表2

In [ ]:
def Trans(r):
  return H_3d(angle=0, axis=[1,0,0], r=r) #axis の方向は任意

Trans([1,2,3]) #お試し

In [ ]:
def Rotx(angle):
  return H_3d(angle=angle, axis=[1,0,0], r=np.zeros(3))

Rotx(pi/3) #お試し

In [ ]:
def Roty(angle):
  return H_3d(angle=angle, axis=[0,1,0], r=np.zeros(3))

Roty(pi/3) #お試し

In [ ]:
def Rotz(angle):
  return H_3d(angle=angle, axis=[0,0,1], r=np.zeros(3))

Rotz(pi/3) #お試し

## マニピュレータの同次変換

### 手先のデータ（今回は最初から同次座標で与える）

In [ ]:
hand = np.array([
  [0,    0,    0,  1],  #[x,y,z,1]　1点目
  [0, -0.2,  0.2,  1],  #2点目
  [0,  0.2,  0.2,  1],  #3点目
  [0,    0,    0,  1]   #4点目 = 1点目
])

### マニピュレータの形状データを求める関数

In [ ]:
def get_manipulator_points( hand, angles ):

  l1, l2, l3 = 0.8, 0.8, 0.8   #リンク長
  th1, th2, th3, th4 = angles  #関節角
  origin = np.array([0,0,0,1]) #原点の同時座標

  # 使用する同次変換行列
  R2 = Rotz( th4 )
  T3 = Trans([0,0,l3])
  R4 = Rotx( -th3 )
  T5 = Trans([0,0,l2])
  R6 = Rotx( -th2 )
  T7 = Trans([0,0,l1])
  R8 = Rotz( th1 )

  # 式(7.11)の形状データの作成

  ##手先　※行列の積np.dot(A,B)は，A.dot(B)という文法でも書けます
  H = R8.dot(T7).dot(R6).dot(T5).dot(R4).dot(T3).dot(R2)
  xi1s = [] #空のリスト
  for point in hand:  #手先
    newpoint = np.dot(H, point)
    xi1s.append(newpoint) #計算結果をリストに追加
  xi1s = np.array(xi1s) #リストを配列化

  ##肘
  H = R8.dot(T7).dot(R6).dot(T5)
  xi4 = np.dot(H, origin)

  ##肩
  H = R8.dot(T7)
  xi5 = np.dot(H, origin)

  ##根本
  xi6 = origin

  ##一筆書きの点列
  allpoints = np.vstack((xi1s, xi4, xi5, xi6))

  ##ダミー成分(最終列)の除去
  allpoints = allpoints[:,:-1]

  return allpoints

get_manipulator_points(hand, [pi/2,pi/2,pi/2,pi/2]) #お試し

## 手動操作

In [ ]:
def show_widgets():
  '''
  手動で図形を動かす
  '''
  def make_slider(name, val, min, max, step):
    spec = {'value':val, 'min':min, 'max':max, 'step':step,
            'discription':name, 'readout_format':'.1f'}
    return widgets.FloatSlider(**spec)

  ths0 = [-40, 50, 30, 90] #初期角度

  th1_input = make_slider('theta 1:', ths0[0], -180, 180, 10)
  th2_input = make_slider('theta 2:', ths0[1], -180, 180, 10)
  th3_input = make_slider('theta 3:', ths0[2], -180, 180, 10)
  th4_input = make_slider('theta 4:', ths0[3], -180, 180, 10)
  output = widgets.Output()

  fig = plt.figure(figsize=(8,8))   #用紙サイズ
  ax = plt.axes(projection='3d')    #用紙上に3次元グラフ用紙を作る
  ax.set_aspect('equal')
  ax.view_init(elev=20) #画角の設定

  @output.capture(clear_output=True, wait=True)
  def draw(th1, th2, th3, th4):

    angles_rad = np.array([th1, th2, th3, th4])/180*pi
    newpoints = get_manipulator_points( hand, angles=angles_rad )
    xs = newpoints[:,0] #1列目　x座標
    ys = newpoints[:,1] #2列目　y座標
    zs = newpoints[:,2] #3列目　z座標

    ax.clear()
    # ax = plt.axes()    #用紙上に3次元グラフ用紙を作る
    ax.set_xlim([-0.5, 2])            #x軸の範囲
    ax.set_ylim([-0.5, 2])            #y軸の範囲
    ax.set_zlim([   0, 2.5])          #z軸の範囲
    ax.set_xlabel('x')                #x軸のラベル
    ax.set_ylabel('y')                #y軸のラベル
    ax.set_zlabel('z')                #x軸のラベル
    ax.grid()

    ax.plot(xs, ys, zs)

    display(ax.figure)

  draw(*ths0) #初期姿勢
  widgets.interactive(draw, th1=th1_input, th2=th2_input, th3=th3_input, th4=th4_input)
  display(th1_input, th2_input, th3_input, th4_input, output)
  plt.close() #余分なプロットを抑制

show_widgets()